This is a tutorial HexWatershed notebook.
This tutorial is an example of the hexwatershed application using a dggrid mesh.

The following publication includes a comprehensive application:

Liao, C., Engwirda, D., Cooper, M., Li, M., and Fang, Y.: Discrete Global Grid System-based Flow Routing Datasets in the Amazon and Yukon Basins, Earth Syst. Sci. Data Discuss. [preprint], https://doi.org/10.5194/essd-2023-398, in review, 2024.

The full documentation of HexWatershed is hosted at: https://hexwatershed.readthedocs.io

In order the run this notebook, you must install the HexWatershed package and its dependencies. 
Besides, the visulization requires the optional dependency packages (see the full documentation installation section).
You can also modify the notebook to use a different visualization method.


First, let's load some Python libraries.

In [1]:
#step 1: load some basic libraries for various operations
import os
import sys
import json
from pathlib import Path
from os.path import realpath
import importlib.util
from shutil import copy2
#check dependencies

iFlag_numpy = importlib.util.find_spec("numpy")
iFlag_gdal = importlib.util.find_spec("osgeo")
iFlag_pyearth = importlib.util.find_spec("pyearth") 
iFlag_pyflowline = importlib.util.find_spec("pyflowline") 
iFlag_geopandas = importlib.util.find_spec("geopandas")

In [2]:
if iFlag_numpy is not None:
    pass
else:
    print("numpy is not installed")

In [3]:
if iFlag_gdal is not None:
    pass
else:
    print("gdal is not installed")

In [4]:
if iFlag_pyearth is not None:
    pass
else:
    print("pyearth is not installed")

In [5]:
if iFlag_geopandas is not None:
    pass
else:
    print("geopandas is not installed")

In [ ]:
if iFlag_pyflowline is not None:
    pass
else:
    print("pyflowline is not installed")

If any dependency is missing, please install it using conda.

In [7]:
#now add the pyflowline into the Python path.
sPath_notebook = Path().resolve()
sPath_parent = str(Path().resolve().parents[0]) 
print(sPath_parent)
#check hexwatershed installation
iFlag_hexwatershed = importlib.util.find_spec("hexwatershed") 
if iFlag_hexwatershed is not None:
    pass
else:
    print('The hexwatershed package is not installed.')
    sys.path.append(sPath_parent)




C:\workspace\python\pyflowline-main
The pyflowline package is not installed. We will use the current path to set it up.


We need to download an additional NetCDF file for this example.
This file is stored on the Github release:
https://github.com/changliao1025/pyflowline/releases/tag/0.2.0
https://github.com/changliao1025/pyflowline/releases/download/0.2.0/lnd_cull_mesh.nc

In [9]:

#add compiled binary into the system path
import os
os.environ["PATH"] += os.pathsep + "/home/jovyan/"

C:\workspace\python\pyflowline-main\data\susquehanna\input\mpas_mesh.nc
File 'C:\workspace\python\pyflowline-main\data\susquehanna\input\mpas_mesh.nc' downloaded successfully.


Now we can import functions within hexwatershed.

In [10]:
#step 3
#load the read configuration function
from pyhexwatershed.change_json_key_value import change_json_key_value
from pyhexwatershed.pyhexwatershed_read_model_configuration_file import pyhexwatershed_read_model_configuration_file

hexwatershed uses a json file for configuration, an example json file is provided.
check whether a configuration exists

In [11]:
sWorkspace_data = realpath( sPath_notebook +  '/data/yukon' )
sWorkspace_input =  str(Path(sWorkspace_data)  /  'input')
sFilename_configuration_in = realpath( sWorkspace_input +  '/pyhexwatershed_amazon_dggrid.json' )
if os.path.isfile(sFilename_configuration_in):
    pass
else:
    print('This configuration does not exist: ', sFilename_configuration_in )

In [12]:

#we can take a look at the content of this json file

with open(sFilename_configuration_in, 'r') as pJSON:
    parsed = json.load(pJSON)
    print(json.dumps(parsed, indent=4))

{
    "sFilename_model_configuration": "/qfs/people/liao313/workspace/python/pyflowline/pyflowline/config/hexwatershed_susquehanna_mpas.json",
    "sWorkspace_data": "/people/liao313/data",
    "sWorkspace_output": "/compyfs/liao313/04model/pyflowline/susquehanna",
    "sWorkspace_project": "/hexwatershed/susquehanna",
    "sWorkspace_bin": "/people/liao313/bin",
    "sRegion": "susquehanna",
    "sModel": "pyflowline",
    "sJob": "hex",
    "iFlag_standalone": 1,
    "iFlag_create_mesh": 1,
    "iFlag_mesh_boundary": 1,
    "iFlag_save_mesh": 1,
    "iFlag_simplification": 1,
    "iFlag_intersect": 1,
    "iFlag_flowline": 1,
    "iFlag_use_mesh_dem": 1,
    "iFlag_global": 0,
    "iFlag_multiple_outlet": 0,
    "iFlag_rotation": 0,
    "iCase_index": 1,
    "iMesh_type": 4,
    "dLongitude_left": -79,
    "dLongitude_right": -74.5,
    "dLatitude_bot": 39.2,
    "dLatitude_top": 42.8,
    "dResolution_degree": 5000,
    "dResolution_meter": 5000,
    "sDate": "20220110",
    "sMesh_

The meaning of these json keywords are explained in the HexWatershed documentation: https://pyflowline.readthedocs.io/en/latest/data/data.html#inputs


Now we can set up some keywords/parameters

In [13]:
#set up some parameters
sMesh_type = 'dggrid' #the dggrid mesh type supported by hexwatershed
sDggrid_type = 'ISEA3H' #a type of dggrid mesh
iCase_index = 1 #a case index for bookmark
iResolution_index = 10 #dggrid resolution index, see dggrid documentation for details.
iFlag_stream_burning_topology = 1 #see hexwatershed documentation for details, also see the publication list.
iFlag_use_mesh_dem = 0
iFlag_elevation_profile = 0

#get today's year, month and day
from datetime import date
today = date.today()
iYear = today.year
iMonth = today.month
iDay = today.day
print("Today's date:", iYear, iMonth, iDay)
sDate = str(iYear) + str(iMonth).zfill(2) + str(iDay).zfill(2) #the date is also a bookmark to label a simulation
sWorkspace_output = sWorkspace_data + '/output/' #this is where the output will be stored

In [14]:
#use this function from pyflowline to check the actual spatial resolution 
from pyflowline.mesh.dggrid.create_dggrid_mesh import dggrid_find_resolution_by_index
dResolution = dggrid_find_resolution_by_index(sDggrid_type, iResolution_index)
print(dResolution)  


In [ ]:
#create a temporal hexwatershed object, later on we will modify several parameters
oPyhexwatershed = pyhexwatershed_read_model_configuration_file(sFilename_configuration_in,
                    iCase_index_in=iCase_index,iFlag_stream_burning_topology_in=iFlag_stream_burning_topology,
                    iFlag_use_mesh_dem_in=0,
                    iFlag_elevation_profile_in=0,
                    iResolution_index_in = iResolution_index, 
                    sDggrid_type_in=sDggrid_type,
                    sDate_in = sDate, sMesh_type_in= sMesh_type)  

#first, we want to change the output directory, even if the json file might be correct, we change it anyway
sWorkspace_output_old = oPyhexwatershed.sWorkspace_output
#we will copy the example configuration files first, so we won't modify the original files
sFilename_configuration_copy= os.path.join( sWorkspace_output, 'pyhexwatershed_configuration_copy.json' )
#copy the main configuration file to the output directory
copy2(sFilename_configuration_in, sFilename_configuration_copy)
#copy the basin configuration file to the output directory as well
sFilename_basins_in = oPyhexwatershed.sFilename_basins
sFilename_configuration_basins_copy = os.path.join( sWorkspace_output, 'pyhexwatershed_configuration_basins_copy.json' )    
copy2(sFilename_basins_in, sFilename_configuration_basins_copy)

#now we can modify these two configuration files without worrying about the original files
sFilename_configuration = sFilename_configuration_copy
sFilename_basins = sFilename_configuration_basins_copy
change_json_key_value(sFilename_configuration_in, 'sWorkspace_output', sWorkspace_output) #output folder
change_json_key_value(sFilename_configuration_in, 'sFilename_basins', sFilename_basins) #basin configuration file

#we want to change the boundary file, which is a geojson file
sFilename_mesh_boundary = realpath(os.path.join(sWorkspace_input, 'boundary_wgs.geojson')) #boundary to clip mesh
change_json_key_value(sFilename_configuration_in, 'sFilename_mesh_boundary', sFilename_mesh_boundary) 




we can now call the function to re-create a hexwatershed object

In [15]:
#the read function accepts several keyword arguments that can be used to change the default parameters.
#the normal keyword arguments are:
#iCase_index_in: this is an ID to identify the simulation case
#sMesh_type_in: this specifies the mesh type ('mpas' in this example)
#sDate_in: this specifies the date of the simulation, the final output folder will have a pattern such as 'pyflowline20230901001', where pyflowline is model, 20230901 is the date, and 001 is the case index.
oPyhexwatershed = None
oPyhexwatershed = pyhexwatershed_read_model_configuration_file(sFilename_configuration,
                    iCase_index_in=iCase_index,iFlag_stream_burning_topology_in=iFlag_stream_burning_topology,
                    iFlag_use_mesh_dem_in=iFlag_use_mesh_dem,
                    iFlag_elevation_profile_in=iFlag_elevation_profile,
                    iResolution_index_in = iResolution_index, 
                    sDggrid_type_in=sDggrid_type,
                    sDate_in= sDate, sMesh_type_in = sMesh_type)  

C:\workspace\python\pyflowline-main\data\susquehanna\output


You can review the setting again.

In [ ]:
print(oPyhexwatershed.tojson())

Other than change the json file directly, you can also change the parameters on the fly for some of the parameters.
This is useful for testing different parameters without changing the json file, especially for individual basins.

In [18]:
#we also need to set the output location for the only basin
dLongitude_outlet_degree=-49.47644
dLatitude_outlet_degree=1.08260
oPyhexwatershed.pPyFlowline.aBasin[0].dThreshold_small_river = dResolution * 5 
oPyhexwatershed.pPyFlowline.pyflowline_change_model_parameter('dLongitude_outlet_degree', dLongitude_outlet_degree, iFlag_basin_in= 1)
oPyhexwatershed.pPyFlowline.pyflowline_change_model_parameter('dLatitude_outlet_degree', dLatitude_outlet_degree, iFlag_basin_in= 1)
#we want to change the flowline file, which is a geojson file
sFilename_flowline = realpath(os.path.join(sWorkspace_input, 'flowline.geojson') )
oPyhexwatershed.pPyFlowline.pyflowline_change_model_parameter('sFilename_flowline_filter', sFilename_flowline, iFlag_basin_in= 1)

You can check the setting for the single basin as well

In [ ]:
print(oPyhexwatershed.aBasin[0].tojson())

After the hexwatershed object was re-created, we can set up the model.

In [ ]:
#setup the model       
oPyhexwatershed.pyhexwatershed_setup()

Before any operation, we can visualize the original or raw flowline dataset. 

In [ ]:
if iFlag_geopandas is not None:
    import geopandas as gpd
    import matplotlib.pyplot as plt  
else:
    print('The visulization packages are not installed.')
pass

You can also use QGIS.

The plot function provides a few optional arguments such as map projection and spatial extent. 
By default, the spatial extent is full. 
But you can set the extent to a zoom-in region.

Now let's run the the submodel pyflowline.

In [ ]:
#run step 1
aCell_origin = oPyhexwatershed.pyhexwatershed_run_pyflowline();

and check the result using a plot

Similarly, we can zoom in using the extent.

Next, we will creata a mesh from the global MPAS mesh.

we can also use a polygon to create a mesh

Last, we can generate the conceptual flowline.

Now we can overlap mesh with flowline.

In [ ]:

oPyhexwatershed.pyhexwatershed_assign_elevation_to_cells()
aCell_new = oPyhexwatershed.pyhexwatershed_update_outlet(aCell_origin)
oPyhexwatershed.pPyFlowline.pyflowline_export()
oPyhexwatershed.pyhexwatershed_export_config_to_json()
oPyhexwatershed.pyhexwatershed_run_hexwatershed()
oPyhexwatershed.pyhexwatershed_analyze()
oPyhexwatershed.pyhexwatershed_export()


After this, we can save the model output into a json file.

In [31]:
#export output

file1_path = oPyhexwatershed.sFilename_mesh
file2_path = oPyhexwatershed.aBasin[0].sFilename_flow_direction
gdf1 = gpd.read_file(file1_path)
gdf2 = gpd.read_file(file2_path)
fig, ax = plt.subplots()
gdf1.plot(ax=ax, color='blue')
gdf2.plot(ax=ax, color='red')
plt.show()
pass

the content of the one of the exported json files can be checked:

The outlet associated flowline is always assigned with a dam, because it would be preserved.

Congratulations! You have successfully finished a pyflowline simulation.